In [65]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Загрузка данных

In [66]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [67]:
data = pd.read_excel('data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [68]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [69]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [70]:
x.drop('lat', axis = 1, inplace = True)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [72]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [73]:
from sklearn.feature_selection import RFE

In [74]:
y = data['price']
x = data.drop(columns='price')

In [75]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [76]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
rfe_columns = selector.get_feature_names_out()

In [77]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [78]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [79]:
from sklearn.feature_selection import SelectKBest, f_regression

In [80]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
skb_columns = selector.get_feature_names_out()

##  Задание 9.5. Модуль ML-6 (HW-03)

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. Загрузите полученный ноутбук (в формате IPYNB) в форму ниже.

КРИТЕРИИ ОЦЕНИВАНИЯ:

1 балл	Верно выделены три столбца-признака для обучения, выбранные RFE.

1 балл	Верно выделены три столбца-признака для обучения, выбранные SelectKBest.

3 балла	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.

3 балла	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.

2 балла	Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

Обучаем модель на признаках, выбранных методом рекурсивного исключения признаков.

In [81]:
rfe_model = LinearRegression()

rfe_model.fit(X_train[rfe_columns], y_train)
rfe_prediction = rfe_model.predict(X_test[rfe_columns])

rfe_r2 = r2_score(y_test, rfe_prediction)

print(f"RFE test R^2: {rfe_r2}")

RFE test R^2: 0.5759285219640942


Обучаем модель на признаках, выбранных методом выбора k лучших переменных.

In [82]:
skb_model = LinearRegression()

skb_model.fit(X_train[skb_columns], y_train)
skb_prediction = skb_model.predict(X_test[skb_columns])

skb_r2 = r2_score(y_test, skb_prediction)

print(f"SKB test R^2: {skb_r2}")

SKB test R^2: 0.6015637873230486


**Результат:** Модель, обученная на переменных, выбранных по методу фильтрации, показала несколько лучший результат, чем модель, обученная на переменных, выбранных методом рекурсивного исключения признаков. Метрика R2 показала результаты 0.60 и 0.58 соответственно. Однако, раз метод рекурсивного исключения всеже применяется, скорее всего на других данных он может показать влучший результат. 
Кроме того метод фильтрации сложнее к применению, из-за сложности выбора статистических показателей, а рекурсивный метож затратнее по времени, так как производится последовательное обучение нескольких моделей.